In [8]:
import geopandas as gpd
import xarray as xr
import rioxarray
import rasterio
from rasterio import features
import numpy as np
from tqdm.autonotebook import tqdm

damage = gpd.read_file("../data_working/damage_merged.gdb/", layer="merged")
template = xr.open_dataset("../mort_datasets/westmort.nc").isel(time=0)

/tmp/ipykernel_120/4131621254.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [20]:
# Determine raster shape/transforms at coarse and fine resolution
template_shape = template.mortality.shape
template_transform = template.mortality.rio.transform()

# Ratio of coarse resolution to fine resolution
resample_factor = 20

fine_shape = tuple(s*resample_factor for s in template_shape)
fine_transform = template_transform * template_transform.scale(1/resample_factor)

In [21]:
def rasterize_fine_coarse(geoms, fine_shape, fine_transform, coarse_shape, coarse_transform):
    fine_raster = features.rasterize(
        geoms,
        out_shape=fine_shape,
        transform=fine_transform,
        fill=0,
        default_value=100,
        all_touched=True,
        dtype=np.byte
    )

    out_array = np.zeros(coarse_shape)

    rasterio.warp.reproject(
        fine_raster,
        destination=out_array,
        src_transform=fine_transform,
        src_crs=3857,
        dst_transform=coarse_transform,
        dst_crs=3857,
        resampling=rasterio.enums.Resampling.average
    )

    return out_array

In [26]:
%%time
def annual_iou(y1, y2):
    # Select two years
    a = damage[damage.SURVEY_YEAR == y1][["HOST_CODE", "geometry"]]
    b = damage[damage.SURVEY_YEAR == y2][["HOST_CODE", "geometry"]]
    
    # Get overall union
    union = gpd.overlay(a, b, how="union")
    
    # Get intersections where host codes match
    intersect = union[(union.HOST_CODE_1 == union.HOST_CODE_2) & (union.HOST_CODE_1 != np.nan)]
    
    # Rasterize both
    intersect_rast = rasterize_fine_coarse(intersect.geometry, fine_shape, fine_transform, template_shape, template_transform)
    union_rast     = rasterize_fine_coarse(union.geometry, fine_shape, fine_transform, template_shape, template_transform)
    
    # Get ratio
    assert intersect_rast.shape == union_rast.shape
    iou = intersect_rast / union_rast
    return iou

test_iou = annual_iou(2002, 2003)

<timed exec>:7: UserWarning: `keep_geom_type=True` in overlay resulted in 92 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries


CPU times: user 25.7 s, sys: 84.9 ms, total: 25.8 s
Wall time: 25.8 s


<timed exec>:18: RuntimeWarning: invalid value encountered in divide


In [29]:
import datetime

unique_years = damage.SURVEY_YEAR.unique()
years_to_process = list(range(2001, max(unique_years)-1))
print(years_to_process)

iou_cube = np.zeros((len(years_to_process), *template_shape))

print(f"Starting at {datetime.datetime.now().strftime('%H:%M:%S')}")

for i, yr in enumerate(tqdm(years_to_process)):
    iou_cube[i] = annual_iou(yr, yr+1)

[2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
Starting at 21:20:02


  0%|          | 0/22 [00:00<?, ?it/s]

<timed exec>:7: UserWarning: `keep_geom_type=True` in overlay resulted in 5 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
<timed exec>:18: RuntimeWarning: invalid value encountered in divide
<timed exec>:7: UserWarning: `keep_geom_type=True` in overlay resulted in 92 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
<timed exec>:18: RuntimeWarning: invalid value encountered in divide
<timed exec>:7: UserWarning: `keep_geom_type=True` in overlay resulted in 1 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
<timed exec>:18: RuntimeWarning: invalid value encountered in divide
<timed exec>:7: UserWarning: `keep_geom_type=True` in overlay resulted in 29 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
<timed exec>:18: RuntimeWarning: in

In [38]:
iou_xr = xr.DataArray(
    data=iou_cube,
    dims=("time", "y", "x"),
    coords={"x": template.x, "y": template.y, "time": years_to_process},
    name="iou"
)*100

In [39]:
iou_xr.to_netcdf("../mort_datasets/westmort_iou.nc", encoding=dict(iou={"dtype": "int8", "_FillValue": -128}))